# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that we've discussed some regularization, initialization and optimization techniques, its time to synthesize those concepts into a cohesive modelling pipeline.  

With this pipeline, yoiu will not only fit an initial model but will also attempt to set various hyperparameters for regularization techniques. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modelling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. We've discussed the bias variance tradeoff, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Implement a K-folds cross validation modelling pipeline
* Apply normalization as a preprocessing technique
* Apply regularization techniques to improve your model's generalization
* Choose an appropriate optimization strategy 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras import optimizers

Using TensorFlow backend.


## Loading the Data

In [2]:
#Your code here; load and preview the dataset
raw_df = pd.read_csv('loan_final.csv')
raw_df.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 16 columns):
loan_amnt              42535 non-null float64
funded_amnt_inv        42535 non-null float64
term                   42535 non-null object
int_rate               42535 non-null object
installment            42535 non-null float64
grade                  42535 non-null object
emp_length             41423 non-null object
home_ownership         42535 non-null object
annual_inc             42531 non-null float64
verification_status    42535 non-null object
loan_status            42535 non-null object
purpose                42535 non-null object
addr_state             42535 non-null object
total_acc              42506 non-null float64
total_pymnt            42535 non-null float64
application_type       42535 non-null object
dtypes: float64(6), object(10)
memory usage: 5.2+ MB


In [4]:
df = raw_df.dropna()
df[['home_ownership', 'verification_status', 'emp_length']] = df[['home_ownership', 'verification_status', 'emp_length']].astype('category')
df.info()

C:\Users\david_000\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41394 entries, 0 to 42516
Data columns (total 16 columns):
loan_amnt              41394 non-null float64
funded_amnt_inv        41394 non-null float64
term                   41394 non-null object
int_rate               41394 non-null object
installment            41394 non-null float64
grade                  41394 non-null object
emp_length             41394 non-null category
home_ownership         41394 non-null category
annual_inc             41394 non-null float64
verification_status    41394 non-null category
loan_status            41394 non-null object
purpose                41394 non-null object
addr_state             41394 non-null object
total_acc              41394 non-null float64
total_pymnt            41394 non-null float64
application_type       41394 non-null object
dtypes: category(3), float64(6), object(7)
memory usage: 4.5+ MB


## Defining the Problem

Set up the problem by defining X and Y. 

For this problem use the following variables for X:
* loan_amnt
* home_ownership
* funded_amnt_inv
* verification_status
* emp_length
* installment
* annual_inc

Be sure to use dummy variables for categorical variables and to normalize numerical quanitities. Be sure to also remove any rows with null data.  

For Y, we are looking to build a model to predict the total payment received for a loan.

In [33]:
# Your code here; appropriately define X and Y using dummy variables and normalization for preprocessing.
X = df[['loan_amnt', 'home_ownership', 'funded_amnt_inv', 'verification_status', 'emp_length', 'installment', 'annual_inc']]
Y = df['total_pymnt'].apply(lambda x: (x - df['total_pymnt'].mean())/df['total_pymnt'].std())

In [34]:
X = pd.get_dummies(X)
num_vars = ['loan_amnt','funded_amnt_inv','installment','annual_inc']
for var in num_vars:
    mean = X[var].mean()
    std = X[var].std()
    X[var] = X[var].apply(lambda x: (x - mean)/std)
X.head()

,loan_amnt,funded_amnt_inv,installment,annual_inc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
0,-0.831054,-0.731187,-0.774536,-0.708851,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,-1.168162,-1.077694,-1.267109,-0.615902,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,-1.181646,-1.091694,-1.149989,-0.890846,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,0
3,-0.156839,-0.027675,0.068919,-0.318464,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,-1.100740,-1.007693,-1.229057,0.158675,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modelling decisions. As such, pull out a sample of 10% of the total available data. For consistency of results, use random seed 123. 

In [35]:
print(Y.head())
len(Y)


0   -0.685754
1   -1.218024
2   -0.999442
3    0.013389
4   -0.882941
Name: total_pymnt, dtype: float64


41394

In [36]:
# Your code here; generate a hold out test set for final model evaluation. Use random seed 123.
import random
random.seed(123)
test_index = random.sample(range(0,len(X)), int(len(X)/10))
train_index = list(set(range(0,len(X))) - set(test_index))

X = X.reset_index(drop = True)
Y = Y.reset_index(drop = True)

X_test = X.loc[test_index, :]
X_train = X.loc[train_index, :]

y_test = Y.loc[test_index]
y_train = Y.loc[train_index]

print("Test label shape:", np.shape(y_test))
print("Train label shape:", np.shape(y_train))
print("Test shape:", np.shape(X_test))
print("Train shape:", np.shape(X_train))

Test label shape: (4139,)
Train label shape: (37255,)
Test shape: (4139, 23)
Train shape: (37255, 23)


## Defining a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, write a function that takes in the remaining data and performs k-folds cross validation given a model object. Be sure your function returns performance metrics regarding the training and validation sets.

In [37]:
#Your code here; define a function to evaluate a model object using K folds cross validation.

def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
    n = len(features_train)
    
    #Get cut points
    cuts = []
    prop = 0
    for i in range(0, n+1):
        if i/n >= prop:
            cuts.append(i)
            prop += 1./k
    
    loss_values = []
    val_loss_values = []
    acc_values = []
    val_acc_values = []
    results_train = []
    results_val = []
    
    for fold in range(0, k):
        val_index = list(range(cuts[fold], cuts[fold+1]))
        train_index = list(set(range(0,n)) - set(val_index))
        
        X_cftrain = features_train.iloc[train_index, :].values
        X_cfval = features_train.iloc[val_index, :].values
        
        y_cftrain = labels_train.iloc[train_index].values
        y_cfval = labels_train.iloc[val_index].values
        
        model_val = model_obj.fit(X_cftrain, y_cftrain, batch_size=128, 
                 epochs=n_epochs, validation_data = (X_cfval, y_cfval), verbose=0)
                
        model_val_dict = model_val.history
        
        #loss_values.append(model_val_dict['loss'])
        #val_loss_values.append(model_val_dict['val_loss'])
        #acc_values.append(model_val_dict['acc']) 
        #val_acc_values.append(model_val_dict['val_acc'])
        results_train.append(model_obj.evaluate(X_cftrain, y_cftrain))
        results_val.append(model_obj.evaluate(X_cfval, y_cfval))
        
    #loss_values = np.mean(loss_values, axis = 1)
    #val_loss_values = np.mean(val_loss_values, axis = 1)
    #acc_values = np.mean(acc_values, axis = 1)
    #val_acc_values = np.mean(val_acc_values, axis = 1)
    
    history = {'loss':loss_values, 'val_loss': val_loss_values, 'acc': acc_values, 'val_acc': val_acc_values}
    
    return results_train, results_val, history
        

## Building a Baseline Model

Here, it is also important to define your evaluation metric that you will look to optimize while tuning the model.   

In general, model training to optimize this metric may consist of using a validation and test set if data is plentiful, or k-folds cross-validation if data is limited. We set up a k-folds cross-validation for this task since the dataset is not overly large.  

Build an initial sequential model with 2 hidden relu layers. The first should have 7 hidden units, and the second 10 hidden units. Finally, add a third layer with a linear activation function to output our predictions for the total loan payment. 

In [44]:
#Your code here; define and compile an initial model as described
model = models.Sequential()
model.add(layers.Dense(7, activation='relu', input_shape=(23,))) #2 hidden layers
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='SGD',
              loss='mean_squared_error')

## Evaluating the Baseline Model with K-Folds Cross Validation

Use your k-folds function to evaluate the baseline model.  

Note: This code block is likely to take 10-20 minutes to run depending on the specs on your computer.
Because of time dependencies, it can be interesting to begin timing these operations for future reference.

Here's a simple little recipe to achieve this:
```
import time
import datetime

now = datetime.datetime.now()
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
```

In [46]:
#Your code here; use your k-folds function to evaluate the baseline model.
import time
import datetime

now = datetime.datetime.now()
results_train, results_val, history = k_folds(X_train, y_train, model, k=10, n_epochs=10)
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
print(np.mean(results_train, axis = 0), np.mean(results_val, axis = 0))

3725/3725 [==============================] - 0s 23us/step
Time Elapsed: 0:01:00.304336
0.18908092044022087 0.19037520430535176


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.  

For example, here are some timing notes on potential experiments run on a Macbook Pro 3.1 GHz Intel Core i5 with 16gb of RAM:

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 100 epochs took approximately 15 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 25 units per hidden layer and 100 epochs took approximately 25 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 250 epochs took approximately 45 minutes to train and validate


In [47]:
#Your code here; try some methods to overfit your network
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=(23,))) #2 hidden layers
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='SGD',
              loss='mean_squared_error')

now = datetime.datetime.now()
results_train, results_val, history = k_folds(X_train, y_train, model, k=10, n_epochs=100)
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
print(np.mean(results_train, axis = 0), np.mean(results_val, axis = 0))
results_test = model.evaluate(X_test, y_test)
print(results_test)


3725/3725 [==============================] - 0s 20us/step
Time Elapsed: 0:11:26.882595
0.1796973363023754 0.18565136590907527


In [48]:
#Your code here; try some methods to overfit your network
results_test = model.evaluate(X_test, y_test)
results_test

4139/4139 [==============================] - 0s 30us/step


0.1947457827240639

In [ ]:
#Your code here; try some methods to overfit your network


## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least 2 different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

Notes: 

In [50]:
#Your code here; try some regularization or other methods to tune your network
model = models.Sequential()
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu', input_shape=(23,))) #2 hidden layers
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='SGD',
              loss='mean_squared_error')

now = datetime.datetime.now()
results_train, results_val, history = k_folds(X_train, y_train, model, k=10, n_epochs=100)
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
print(np.mean(results_train, axis = 0), np.mean(results_val, axis = 0))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
3725/3725 [==============================] - 0s 27us/step
Time Elapsed: 0:18:41.028341
0.7156636421170197 0.7088419322057029
4139/4139 [==============================] - 0s 27us/step
0.7732957356446721


In [52]:
#Your code here; try some regularization or other methods to tune your network
#Your code here; try some methods to overfit your network
from keras import regularizers
model = models.Sequential()
model.add(layers.Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu', input_shape=(23,))) #2 hidden layers
model.add(layers.Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='SGD',
              loss='mean_squared_error')

now = datetime.datetime.now()
results_train, results_val, history = k_folds(X_train, y_train, model, k=10, n_epochs=100)
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
print(np.mean(results_train, axis = 0), np.mean(results_val, axis = 0))
results_test = model.evaluate(X_test, y_test)
print(results_test)


3725/3725 [==============================] - 0s 42us/step
Time Elapsed: 1:37:56.535286
0.22297216872905107 0.22739897310010204
4139/4139 [==============================] - 0s 32us/step
0.2261014519595501


In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your finalized model once and for all. Fit the model using all of the training and validation data using the architecture and hyperparameters that were most effective in your expirements above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modelling process)!

In [53]:
#Your code here; final model training on entire training set followed by evaluation on hold-out data
#Your code here; define and compile an initial model as described
model = models.Sequential()
model.add(layers.Dense(7, activation='relu', kernel_regularizer=regularizers.l1(0.005), input_shape=(23,))) #2 hidden layers
model.add(layers.Dense(10, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
model.add(layers.Dense(1, kernel_regularizer=regularizers.l1(0.005), activation='linear'))
model.compile(optimizer='SGD',
              loss='mean_squared_error')

now = datetime.datetime.now()
results_train, results_val, history = k_folds(X_train, y_train, model, k=10, n_epochs=100)
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
print(np.mean(results_train, axis = 0), np.mean(results_val, axis = 0))

results_test = model.evaluate(X_test, y_test)
print(results_test)

3725/3725 [==============================] - 0s 38us/step
Time Elapsed: 0:10:34.621633
0.21209594692771194 0.2155695429658783
4139/4139 [==============================] - 0s 35us/step
0.2163967914585907


## Additional Resources

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
https://www.springboard.com/blog/free-public-data-sets-data-science-project/

## Summary

In this lab, we investigated some data from *The Lending Club* in a complete data science pipeline regarding neural networks. We began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, we implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. From here, we'll begin to investigate other neural network architectures such as CNNs.